$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool
from ROOT import TCanvas
import matplotlib.pyplot as plt

## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")
from Uniandes_Framework.delphes_reader.lhereader import LHE_Loader, readLHEF, get_event_by_child
from Uniandes_Framework.delphes_reader.root_analysis import get_kinematics_row, make_histograms, overlap_histos, Quiet, generate_csv, sum_histos, histos_matplotlib

Welcome to JupyROOT 6.22/06


hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.


Already up to date.


In [2]:
Folder = 'Data_SDE_1'

Betards = ['woRHC']
zp_limits = ['zp_upper_limit', 
             'zp_lower_limit']

In [3]:
generation_options = {'ta_ta_virtual_contribution':  'generate p p > ta ta QED = 0 $$ zp', 
                      'ta_ta_real_contribution': 'generate p p > zp > ta ta QED = 0',
                      'ta_ta_total_contribution': 'generate p p > ta ta QED = 0'}

In [4]:
M_Us = np.arange(1000,3500,500)
g_Us = np.arange(0.5,1.5,0.5)
g_Zps = np.arange(0.5,2,0.5)

# M_Us = [1500]
# g_Us = [1.0]
# g_Zps = [0.5]

In [5]:
def read_xs_from_html(signal, mu_gu_gzp_label, Betard, zp_limit):
    html_path = os.path.join(os.getcwd(), Folder, Betard, zp_limit, signal,  mu_gu_gzp_label, 'crossx.html')
    html_table = pd.read_html(html_path)
    column_xs = html_table[0]['Cross section (pb)']
    return float(column_xs[0].split(' ')[0])    

csv_initial = pd.read_csv(os.path.join('Uniandes_Framework', 'SimulationsPaths.csv'))

csv_final = pd.concat(
                        [csv_initial]
                        +
                        [ pd.DataFrame.from_dict(
                        {
                            'name': f'{Betard}_{zp_limit}_{signal}_mu_{m}_gu_{gu}_gzp_{gzp}', 
                            'path': os.path.join(os.getcwd(), Folder, Betard, zp_limit, signal, f'mu_{m}_gu_{gu}_gzp_{gzp}'),  
                            'xs(pb)': read_xs_from_html(signal, f'mu_{m}_gu_{gu}_gzp_{gzp}', Betard, zp_limit)
                        }, 
                        orient = 'index').T for signal, m , gu, gzp, Betard, zp_limit in product(generation_options.keys(),M_Us,g_Us, g_Zps, Betards, zp_limits)]
                     )

# save the new csv
csv_final.to_csv('SimulationsPaths.csv', index = False)

In [6]:
def read_lhe_file(signal):
    global label, Betard, zp_limit, Folder
    
    signal_key = f'{Betard}_{zp_limit}_{signal}_{label}'
    
    lhe_loader = LHE_Loader(signal_key, os.path.join(os.getcwd(), 'SimulationsPaths.csv'))
    lhe_path_files = lhe_loader.Forest
        
    results = list()
    
    for lhe_path_file in lhe_path_files:
    
        childs=readLHEF(lhe_path_file)

        for child in childs:
            event = get_event_by_child(child)
            taus = event.getParticlesByIDs([-15,15])
            if len(taus) != 2: continue
            taus[0].SetName("#tau_{1}")
            taus[1].SetName("#tau_{2}")
            sum_taus = taus[0].TLV + taus[1].TLV
            row = get_kinematics_row(taus)
            row["m_ll(GeV)"] = sum_taus.M()
            row["sT(GeV)"] = row["pT_{#tau_{1}}(GeV)"] + row["pT_{#tau_{2}}(GeV)"]
            results.append(row)        
    
    generate_csv(results, os.path.join(Folder, Betard, zp_limit, signal, label, 'Kinematic_Information.csv'))

In [7]:
generation_options = {'ta_ta_virtual_contribution':  'generate p p > ta ta QED = 0 / zp', 
                      'ta_ta_real_contribution': 'generate p p > zp, zp > ta ta QED = 0',
                      'ta_ta_total_contribution': 'generate p p > ta ta QED = 0'}

In [ ]:
for mu , gu, gzp, Betard, zp_limit in product(M_Us,g_Us, g_Zps, Betards, zp_limits):
    
    label = f'mu_{mu}_gu_{gu}_gzp_{gzp}'
    if not os.path.exists(os.path.join(Folder, Betard, zp_limit, 'ta_ta_virtual_contribution', label, 'Kinematic_Information.csv')):
        with Pool(3) as p: p.map(read_lhe_file, generation_options.keys())

woRHC_zp_upper_limit_ta_ta_real_contribution_mu_3000_gu_1.0_gzp_1.0 imported with 6 trees!
/home/pheno/2023/Semanas_22_23_24/lq_zprime/01_signal_production/ta_ta_generation_testing/Data_SDE_1/woRHC/zp_upper_limit/ta_ta_real_contribution/mu_3000_gu_1.0_gzp_1.0
woRHC_zp_upper_limit_ta_ta_total_contribution_mu_3000_gu_1.0_gzp_1.0 imported with 6 trees!
/home/pheno/2023/Semanas_22_23_24/lq_zprime/01_signal_production/ta_ta_generation_testing/Data_SDE_1/woRHC/zp_upper_limit/ta_ta_total_contribution/mu_3000_gu_1.0_gzp_1.0woRHC_zp_upper_limit_ta_ta_virtual_contribution_mu_3000_gu_1.0_gzp_1.0 imported with 6 trees!
/home/pheno/2023/Semanas_22_23_24/lq_zprime/01_signal_production/ta_ta_generation_testing/Data_SDE_1/woRHC/zp_upper_limit/ta_ta_virtual_contribution/mu_3000_gu_1.0_gzp_1.0

woRHC_zp_lower_limit_ta_ta_total_contribution_mu_3000_gu_1.0_gzp_1.0 imported with 6 trees!
/home/pheno/2023/Semanas_22_23_24/lq_zprime/01_signal_production/ta_ta_generation_testing/Data_SDE_1/woRHC/zp_lower_limi

In [ ]:
!touch acabo_Generate_CSVs.txt